In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from collections import deque
import copy
import time

In [16]:
import numpy as np


# all array inputs must be of the type np.array

class my_env:
    #def __init__(self, uav_number, user_number, size, user_loc, noise, height, transmit_pow, safe_dist): # initializes environment
    def __init__(self):
        self.uav_number = 2
        self.user_number = 5
        self.size = np.array([10,10])
        self.user_loc = np.array([[-5,-5,5,5,0],[-5,5,-5,5,0]])
        self.safe_dist = 5
        self.noise = 0.01
        self.height = 7
        self.transmit_pow = 1
        self.observation_space = 2*self.uav_number + self.user_number

        ### Parameters
        self.actions = 5 # 0: 10 units Left , 1: 10 units up , 2: 10 units right, 3: 10 units down, 4: stay
        self.user_profile = 1*np.ones(self.user_number)
        ###

    def my_reward(self, uav_loc, scheduling, user_prof, agent): # calculate reward
        num_agents = np.shape(uav_loc)[1]
        num_user = np.shape(self.user_loc)[1]

        pref = np.zeros((num_agents,num_user))
        pref_rate = np.zeros((num_agents,num_user))
        for i in range(np.shape(pref)[0]):
            for j in range(np.shape(pref)[1]):
                interf = 0
                for k in range(np.shape(pref)[0]):
                    if k != i:
                        interf = interf + self.transmit_pow/(self.height**2 + np.linalg.norm(uav_loc[:,k]-self.user_loc[:,j]))
                pref[i,j] = user_prof[j]*np.log2(1 + self.transmit_pow/(self.height**2 + np.linalg.norm(uav_loc[:,i]-self.user_loc[:,j]))/(interf+self.noise))
                #pref[i,j] = np.log2(1 + self.transmit_pow/(self.height**2 + np.linalg.norm(uav_loc[:,i]-self.user_loc[:,j]))/(interf+self.noise))


        ####### ---- penalty
        penalty = 0

        for i in range(np.shape(pref)[0]):
            for k in range(np.shape(pref)[0]):
                if k != i:
                    penalty = penalty + (np.linalg.norm(uav_loc[:,k]-uav_loc[:,i]) < self.safe_dist)

        # reward = np.sum(np.multiply(scheduling,pref)) - np.log2(1 + self.transmit_pow/(self.height**2)/self.noise )*penalty/2
        reward = np.sum(np.multiply(scheduling[agent-1,:],pref[agent-1,:])) - 1000*penalty/4

        if penalty != 0:
            done =  True
        else:
            done = False

        return reward , done


    def next_state(self, action1, uav_loc, user_prof, scheduling):
        if action1<5:
            action = np.base_repr(action1,base=5,padding = 1)
        else :
            action = np.base_repr(action1,base=5,padding = 0)

        for i in range(len(action)):
            if action[i] == "0":

                uav_loc[:,i] = uav_loc[:,i] + (-2*np.array([1,0]))
                uav_loc[0,i] = uav_loc[0,i] + (uav_loc[0,i]<-self.size[0]/2)*(-self.size[0]/2-uav_loc[0,i])

            elif action[i] == "1":

                uav_loc[:,i] = uav_loc[:,i] + (2*np.array([0,1]))
                uav_loc[1,i] = uav_loc[1,i] + (uav_loc[1,i]>self.size[1]/2)*(self.size[1]/2-uav_loc[1,i])

            elif action[i] == "2":

                uav_loc[:,i] = uav_loc[:,i] + (2*np.array([1,0]))
                uav_loc[0,i] = uav_loc[0,i] + (uav_loc[0,i]>self.size[0]/2)*(self.size[0]/2-uav_loc[0,i])

            elif action[i] == "3":

                uav_loc[:,i] = uav_loc[:,i] + (-2*np.array([0,1]))
                uav_loc[1,i] = uav_loc[1,i] + (uav_loc[1,i]<-self.size[1]/2)*(-self.size[1]/2-uav_loc[1,i])

            else :
                pass

        #user_prof = user_prof + (0.1 * ~(np.sum(scheduling,axis=0)>0))
        # user_prof = user_prof - (np.min(user_prof)>5)*5
        user_prof = 1/(0.9/user_prof + 0.1*np.sum(scheduling,axis=0))

        return np.hstack((uav_loc.flatten() , user_prof))

    def init_state(self):

        while True:
            a = np.random.uniform(-self.size[0]/2,self.size[0]/2,size = (1,self.uav_number))
            b = np.random.uniform(-self.size[1]/2,self.size[1]/2,size = (1,self.uav_number))
            c = np.vstack((a,b))
            if np.linalg.norm(c[:,0]-c[:,1])>self.safe_dist:
                break
        
 

        return np.hstack((a.flatten(),b.flatten(),self.user_profile))



import cvxpy as cp
import numpy as np


def schedule(user_prof,user_loc,UAV_loc,transmit_pow,noise,height):


    num_agents = np.shape(UAV_loc)[1]
    num_user = np.shape(user_loc)[1]

    A = cp.Variable((num_agents,num_user))

    pref = np.zeros((num_agents,num_user))
    for i in range(np.shape(pref)[0]):
        for j in range(np.shape(pref)[1]):
            interf = 0
            for k in range(np.shape(pref)[0]):
                if k != i:
                    interf = interf + transmit_pow/(height**2 + np.linalg.norm(UAV_loc[:,k]-user_loc[:,j]))
            pref[i,j] = user_prof[j]*np.log2(1 + transmit_pow/(height**2 + np.linalg.norm(UAV_loc[:,i]-user_loc[:,j]))/(interf+noise))

    objective = cp.Minimize(-1*cp.sum(cp.multiply(A,pref)))

    constraints = [A>=0,A<=1]
    constraints.append(cp.sum(A,axis = 0) <= 1)
    constraints.append(cp.sum(A,axis = 1) <= 1)

    problem = cp.Problem(objective, constraints)

    problem.solve()

    return np.round(A.value)

In [17]:


# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#device = "cpu"

Using device: cuda


In [18]:
# Hyperparameters
GAMMA = 0.99
LR = 0.001
EPISODES = 10000
STEPS_PER_EPISODE = 200

QUAD = my_env()

def weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)

def average_weights(model1, model2):
    """Replace the weights of both models with the average of their weights."""
    for param1, param2 in zip(model1.parameters(), model2.parameters()):
        param1.data.copy_((param1.data + param2.data) / 2)
        param2.data.copy_(param1.data)


#Define Actor network
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 256)
        self.fc2 = nn.Linear(256, action_dim)
        self.softmax = nn.Softmax(dim = -1)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        return (F.softmax(self.fc2(x),dim = -1))




# Define Critic network
class Critic(nn.Module):
    def __init__(self, state_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, 1)  # Output a single value

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        return self.fc4(x)  # State value estimation



In [19]:

# Initialize networks on GPU
actor1 = Actor(QUAD.observation_space,QUAD.actions).to(device)
critic1 = Critic(QUAD.observation_space).to(device)

actor2 = Actor(QUAD.observation_space,QUAD.actions).to(device)
critic2 = Critic(QUAD.observation_space).to(device)

# for param in actor.parameters():
#     if param.ndimension() > 1:  # Check if it's a weight (not a bias)
#         torch.nn.init.constant_(param, 0.0001)

actor_prev1 = Actor(QUAD.observation_space,QUAD.actions).to(device)
critic_prev1 = Critic(QUAD.observation_space).to(device)

actor_prev2 = Actor(QUAD.observation_space,QUAD.actions).to(device)
critic_prev2 = Critic(QUAD.observation_space).to(device)


actor1.apply(weight_init)
critic1.apply(weight_init)

actor2.apply(weight_init)
critic2.apply(weight_init)

# Optimizers
actor_optimizer1 = optim.Adam(actor1.parameters(), lr=LR)
critic_optimizer1 = optim.Adam(critic1.parameters(), lr=LR)

actor_optimizer2 = optim.Adam(actor2.parameters(), lr=LR)
critic_optimizer2 = optim.Adam(critic2.parameters(), lr=LR)



In [20]:
def weight_change(model1,model2):
    weight_diff = 0
    for param1, param2 in zip(model1.parameters(), model2.parameters()):
        weight_diff += torch.sum(torch.abs(param1 - param2)).item()
    return weight_diff

In [21]:
tic = time.time()

reward_acul = np.empty(0)

actor_prev1.load_state_dict(actor1.state_dict())
critic_prev1.load_state_dict(critic1.state_dict())

actor_prev2.load_state_dict(actor1.state_dict())
critic_prev2.load_state_dict(critic1.state_dict())


for episode in range(1,EPISODES+1):
    state = QUAD.init_state()  # Get initial state
    total_reward = 0


    for step in range(1,STEPS_PER_EPISODE+1):

        # Select action

        state_tensor = torch.FloatTensor(state).to(device)

        action_probs1= actor1(state_tensor)

        action1 = torch.multinomial(action_probs1, 1).item()  # Sample action

        action_probs2= actor2(state_tensor)

        action2 = torch.multinomial(action_probs2, 1).item()  # Sample action
        
###############################

        uav_loc = np.reshape(state[:-QUAD.user_number], (2,-1))
        user_profile = state[-QUAD.user_number:]

        scheduling = schedule(user_profile,QUAD.user_loc,uav_loc,QUAD.transmit_pow,QUAD.noise,QUAD.height)

        action = 5*action1+action2

        next_state = QUAD.next_state(action,uav_loc,user_profile,scheduling)

        uav_loc = np.reshape(next_state[:-QUAD.user_number], (2,-1))


        reward1 , done1 = QUAD.my_reward(uav_loc, scheduling, user_profile,1)
        reward2 , done2 = QUAD.my_reward(uav_loc, scheduling, user_profile,2)
        reward = reward1 + reward2

        state_tensor = torch.tensor(np.array(state), dtype=torch.float32, device=device)
        action1 = torch.tensor(action1, dtype=torch.int64, device=device)
        action2 = torch.tensor(action2, dtype=torch.int64, device=device)
        reward_tensor1 = torch.tensor(reward1, dtype=torch.float32, device=device)
        reward_tensor2 = torch.tensor(reward2, dtype=torch.float32, device=device)
        next_tensor = torch.tensor(np.array(next_state), dtype=torch.float32, device=device)


################################ training

        # Compute Advantage
        value1 = critic1(state_tensor)
        value2 = critic2(state_tensor)
        next_value1 = critic1(next_tensor).detach()
        next_value2 = critic2(next_tensor).detach()
        advantage1 = reward_tensor1 + (GAMMA * next_value1 * (not done1)) - value1
        advantage2 = reward_tensor2 + (GAMMA * next_value2 * (not done2)) - value2

        # Update Actor 1
        log_prob1 = torch.log(action_probs1[action1])
        loss_actor1 = -log_prob1 * advantage1.detach()
        actor_optimizer1.zero_grad()
        loss_actor1.backward()
        actor_optimizer1.step()

        # Update Actor 2
        log_prob2 = torch.log(action_probs2[action2])
        loss_actor2 = -log_prob2 * advantage2.detach()
        actor_optimizer2.zero_grad()
        loss_actor2.backward()
        actor_optimizer2.step()

        # Update Critic 1
        loss_critic1 = advantage1.pow(2)
        critic_optimizer1.zero_grad()
        loss_critic1.backward()
        critic_optimizer1.step()

        # Update Critic 2
        loss_critic2 = advantage2.pow(2)
        critic_optimizer2.zero_grad()
        loss_critic2.backward()
        critic_optimizer2.step()

        # consensus on critics 

        average_weights(critic1, critic2)

################################ end of training


        # Move to next state
        state = next_state

        total_reward += reward


        if done1:
            break


    reward_acul = np.append(reward_acul,total_reward)

    if episode%10 == 0 :
        toc = time.time()
        print(f"episode {episode}/{EPISODES}, mean past 10 Reward: {np.mean(reward_acul[-10:])},actor1 weights: {weight_change(actor1,actor_prev1)}, actor2 weights: {weight_change(actor2,actor_prev2)}, critic weights: {weight_change(critic1,critic_prev1)},  elapsed: {(toc-tic)/60} minutes")
        actor_prev1.load_state_dict(actor1.state_dict())
        critic_prev1.load_state_dict(critic1.state_dict())

        actor_prev2.load_state_dict(actor2.state_dict())
        critic_prev2.load_state_dict(critic2.state_dict())

    if episode%100 == 0:

        check_point = {
        'actor1_state':actor1.state_dict(),
        'actor1_optimizer': actor_optimizer1.state_dict(),
        'critic1_state':critic1.state_dict(),
        'critic1_optimizer': critic_optimizer1.state_dict(),
        'actor2_state':actor2.state_dict(),
        'actor2_optimizer': actor_optimizer2.state_dict(),
        'critic2_state':critic2.state_dict(),
        'critic2_optimizer': critic_optimizer2.state_dict(),
        'episode' : episode
        }

        torch.save(check_point,"model.pth")
        np.save("reward.npy",reward_acul)
        np.save(f"episode.npy",episode)



episode 10/10000, mean past 10 Reward: -604.7299235417083,actor1 weights: 43.35863198339939, actor2 weights: 421.59405583143234, critic weights: 40864.04282869212,  elapsed: 0.15850316286087035 minutes
episode 20/10000, mean past 10 Reward: 250.42431800349337,actor1 weights: 59.48215910047293, actor2 weights: 66.23610829561949, critic weights: 8164.5203276611865,  elapsed: 0.6094334999720256 minutes
episode 30/10000, mean past 10 Reward: 249.14452361671988,actor1 weights: 62.478391490876675, actor2 weights: 62.539916314184666, critic weights: 12371.877602469176,  elapsed: 1.0382891813913981 minutes
episode 40/10000, mean past 10 Reward: 460.8278914645737,actor1 weights: 55.46295992285013, actor2 weights: 0.021109493682160974, critic weights: 7191.372971333563,  elapsed: 1.5585967183113099 minutes
episode 50/10000, mean past 10 Reward: 769.1567894984844,actor1 weights: 2.208088904619217, actor2 weights: 0.06942390091717243, critic weights: 2291.01672886312,  elapsed: 2.160538677374522 m

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

a = np.load("reward.npy")

# Define the window size for the moving average
window_size = 10

# Calculate the moving average using pandas' rolling method
moving_avg = pd.Series(a).rolling(window=window_size).mean()

# Convert the result back to a NumPy array (if needed)
moving_avg_np = moving_avg.to_numpy()

plt.figure(figsize=(30, 8))

plt.plot(moving_avg_np)

plt.show